Steps
- Cleaning text
- Word Tokenization
- Word frequency count
- Normalize the word frequency count
- Sentence Tokenization
- Sentence score calculation : by adding up word frequency count for each word
- get 30% of the sentences  

Connecting to Google Drive

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

     |████████████████████████████████| 6.0 MB 5.6 MB/s 
     |████████████████████████████████| 457 kB 32.0 MB/s 
     |████████████████████████████████| 10.1 MB 15.0 MB/s 
     |████████████████████████████████| 653 kB 34.6 MB/s 
     |████████████████████████████████| 181 kB 25.5 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.

In [ ]:
# !pip install google-search-results

  Created wheel for google-search-results: filename=google_search_results-2.4.1-py3-none-any.whl size=25789 sha256=9e5bdc472a5eecc067c4c8e254d21c150ee51cbd2116177d08109001bf343616
  Stored in directory: /root/.cache/pip/wheels/82/a3/c5/364155118f298722dff2f79ae4dd7c91e92b433ad36d6f7e0e
Successfully built google-search-results


In [ ]:
# !pip install pdfminer.six

     |████████████████████████████████| 5.6 MB 5.2 MB/s 
     |████████████████████████████████| 3.6 MB 31.3 MB/s 


In [ ]:
# !pip install rouge

In [ ]:
import spacy
from heapq import nlargest
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from serpapi import GoogleSearch
import os, urllib.request
import nltk
import re
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import sys, getopt
# from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from PIL import Image
# nltk.download("stopwords")
# nltk.download("punkt")

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from rouge import Rouge

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Downloading pdfs from google search engine

In [ ]:
def get_pdfs():
  search = GoogleSearch({
        "api_key": "3948c6d276b1a902335d80dc905d6cb45b14d7056bb017ddbe00a5b01b9d648b",
        "engine": "google",
        "q": "wildfire detection arxiv:pdf",
        "hl": "en"
      })

  results = search.get_dict()
  count = 0

  for index, result in enumerate(results['organic_results']):
        if 'pdf' in result['link']:
          pdf_file = result['link']

          opener=urllib.request.build_opener()
          opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582')]
          urllib.request.install_opener(opener)

          # save PDF
          if count < 3:
            urllib.request.urlretrieve(pdf_file, f"/content/drive/MyDrive/data240/Data 240 Project/final_text_summarization/data/pdf_file_{index}.pdf")
            count += 1
            print(f'Saving PDF №{index}..')
        else: pass


In [ ]:
# get_pdfs()

https://serpapi.com/search
Saving PDF №1..
Saving PDF №2..
Saving PDF №3..


# Converting pdfs into text file

In [ ]:

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 


Combining all files

In [ ]:
#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
def combinePdf(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        if fileExtension == "pdf":
            pdfFilename = pdfDir + pdf 
            text = convert(pdfFilename) #get string of text content of pdf
            textFilename = txtDir + pdf + ".txt"
            textFile = open(textFilename, "w") #make text file
            textFile.write(text) #write text to text file

# # set paths accordingly:
pdfDir = "/content/drive/MyDrive/data240/Data 240 Project/final_text_summarization/data/"
txtDir = "/content/drive/MyDrive/data240/Data 240 Project/final_text_summarization/data/"
combinePdf(pdfDir, txtDir)

In [ ]:
dir = '/content/drive/MyDrive/data240/Data 240 Project/final_text_summarization/data/'

total_text = ""
for foldername,subfolders,files in os.walk(r"/content/drive/MyDrive/data240/Data 240 Project/final_text_summarization/data/"):
  for filename in files:
    if ".txt" in filename:    
      with open(dir+filename) as infile:
          contents = infile.read()
          print(f"Length of the content in file {filename} : {len(contents)}")
          total_text += contents

Length of the content in file pdf_file_1.pdf.txt : 27308
Length of the content in file pdf_file_2.pdf.txt : 34936
Length of the content in file pdf_file_3.pdf.txt : 30158


In [ ]:
print(f"Length of the total contents combined from all files: {len(total_text)}")

Length of the total contents combined from all files: 92402


Helper Functions

In [ ]:
def processText(text):
  processedText = re.sub("’", "'", text)
  processedText = re.sub("[^a-zA-Z' ]+", " ", processedText) 
  return processedText

In [ ]:
def getStopwords():
  stopwords_spacy = list(STOP_WORDS)
  # print(stopwords_spacy)
  return stopwords_spacy

Summerization using spacy and heapq libraries

In [ ]:
def summarizationLib(total_text):
  processedText = processText(total_text)
  # get stop words
  stopwords_spacy = getStopwords()
  # Removing punctuations
  str_punctuation = punctuation + '\n'
  # print(punctuation)
   # Word Tokenization
  spacy_tokenization = spacy.load('en_core_web_sm')
  doc = spacy_tokenization(total_text)
  # tokens = [token.text for token in doc]
  # print(tokens)
  # word frequency count
  word_frequencies = {}
  for word in doc:
    if word.text.lower() not in stopwords_spacy:
      if word.text.lower() not in str_punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
  # normalization of word frequencies
  max_frequency = max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word] / max_frequency

  # Sentence tokenization
  sentence_tokens = [sent for sent in doc.sents]
  # sentence_tokens
  # Sentence scores 
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():
          sentence_scores[sent] = word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent] += word_frequencies[word.text.lower()]
  # get 30% of the sentences with max score 
  select_len = int(len(sentence_tokens)*0.3)
  select_len
  # we need to select select_len number of sentences as part of summary
  summary = nlargest(select_len,sentence_scores,key=sentence_scores.get)
  summary
  # combine sentences together
  final_summary = [word.text for word in summary]
  summary = ' '.join(final_summary)
  # print("Summary using libraries : \n")
  # print(summary)
  return summary



In [ ]:
summary_lib = summarizationLib(total_text)
summary_lib

'T\n\ni\n\n104\n\n103\n\n102\n\nCAL FIRE database\nProposed method\n\n2\n\n4\n\n6\n\n8\n\nNumber of Sensors\n\n10\n105\n\n(a)\n\n104\n\n)\n\n2\nm\nk\n(\na\ne\nr\nA\n\n103\n\n102\n\nCAL FIRE database\n\nProposed method\n\n2\n\n4\n\n6\n\n8\n\nNumber of Sensors\n\n10\n105\n\n(b)\n\n)\nn\no\nt\n(\nn\no\ni\ns\ns\ni\nm\nE\nn\no\nb\nr\na\nC\n\n108\n\n107\n\n106\n\nCAL FIRE database\nProposed method\n\n2\n\n4\n\n6\n\n8\n\nNumber of Sensors\n\n10\n105\n\n(c)\n\n)\n\nD\nS\nU\n\n(\n\ne\nc\ni\nr\nP\nn\no\nb\nr\na\nC\n\n109\n\n108\n\n107\n\nCAL FIRE database\nProposed method\n\n2\n\n4\n\n6\n\n8\n\nNumber of Sensors\n\n10\n105\n\n(d)\n\n)\n\nD\nS\nU\n\n(\ns\ng\nn\ni\nv\na\nS\n\n1.12\n\n1.1\n\n1.08\n\n1.06\n\n1.04\n\n1.02\n\n1\n\n109\n\nPer sensor cost = \n1\n0\n2\n2\n:\nv\ni\nX\nr\na\n\n1\n\n \n \n \n \n \n \n\x0c3\n3\n\n3\n3\n\n4\n4\n5\n5\n6\n6\n7\n7\n7\n7\n\n8\n8\n8\n9\n9\n12\n12\n12\n13\n13\n\n14\n14\n14\n14\n15\n\n15\n\nContents\n\n1 Introduction\n\n1.1 Research Outline . \n\n[19] Climate\n\n”ER

In [ ]:
len(summary_lib)

41069

### Generate Summary using factor based implementation

In [ ]:
def summarize(text):
    # Process text by removing numbers and unrecognized characters
    processedText = processText(text)
    # get stop words
    stopwords_spacy = getStopwords()
    # Word tokenization
    words = word_tokenize(processedText)
    # Normalize words with Porter stemming and build word frequency table
    stemmer = SnowballStemmer("english", ignore_stopwords=True)
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopwords_spacy:
            continue
        elif stemmer.stem(word) in freqTable:
            freqTable[stemmer.stem(word)] += 1
        else:
            freqTable[stemmer.stem(word)] = 1
    # Normalize every sentence in the text
    sentences = sent_tokenize(text)
    stemmedSentences = []
    sentenceValue = dict()
    for sentence in sentences:
        stemmedSentence = []
        for word in sentence.lower().split():
            stemmedSentence.append(stemmer.stem(word))
        stemmedSentences.append(stemmedSentence)
    # Calculate value of every normalized sentence based on word frequency table
    # [:12] helps to save space
    for num in range(len(stemmedSentences)):
        for wordValue in freqTable:
            if wordValue in stemmedSentences[num]:
                if sentences[num][:12] in sentenceValue:
                    sentenceValue[sentences[num][:12]] += freqTable.get(wordValue)
                else:
                    sentenceValue[sentences[num][:12]] = freqTable.get(wordValue)
    # Determine average value of a sentence in the text
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue.get(sentence)
    average = int(sumValues / len(sentenceValue))
    # Create summary of text using sentences that exceed the average value by some factor
    # This factor can be adjusted to reduce/expand the length of the summary
    summary = ""
    for sentence in sentences:
            if sentence[:12] in sentenceValue and sentenceValue[sentence[:12]] > (3.0 * average):
                summary += " " + " ".join(sentence.split())
    
    print(summary)
    return summary

In [ ]:
summary = summarize(total_text)
summary

92402
 Although there are many applications of Machine Learning in the ﬁeld of early wildﬁre detection, many are too costly to implement and/or not feasible to work with. Although there are clouds in the background that could be classiﬁed as smoke, they are not considered as smoke caused by a wildﬁre as they are not coming from the ground. However, these papers have several issues and limitations regarding practicality due to their failure to consider the Machine Learning obstacles. Although the accuracies presented by these papers may seem enticing at ﬁrst glance, they are not valid representations of how well their models would perform when given real-world data from common cameras used for early wild- ﬁre detection, such as the ones provided by ALERTWildﬁre. For instance, one common issue that circulates through the research in this ﬁeld is the issue of Model Robustness–the model’s ability to identify the important features that distinguish one class from another in the case of imag

' Although there are many applications of Machine Learning in the ﬁeld of early wildﬁre detection, many are too costly to implement and/or not feasible to work with. Although there are clouds in the background that could be classiﬁed as smoke, they are not considered as smoke caused by a wildﬁre as they are not coming from the ground. However, these papers have several issues and limitations regarding practicality due to their failure to consider the Machine Learning obstacles. Although the accuracies presented by these papers may seem enticing at ﬁrst glance, they are not valid representations of how well their models would perform when given real-world data from common cameras used for early wild- ﬁre detection, such as the ones provided by ALERTWildﬁre. For instance, one common issue that circulates through the research in this ﬁeld is the issue of Model Robustness–the model’s ability to identify the important features that distinguish one class from another in the case of image rec

In [ ]:
len(summary)

10865

### Evaluation

In [ ]:
human_summary = """Wildfires are uncontrolled fires in the environment that can be caused
by humans or nature. In 2020 alone, wildfires in California have burned
4.2 million acres, damaged 10,500 buildings/structures, and killed more
than 31 people, exacerbated by climate change and a rise in average global
temperatures [1] [2]. This also means there has been an increase in the
costs of extinguishing these treacherous wildfires. The objective of the
research is to detect forest fires in their earlier stages to prevent them
from spreading, prevent them from causing damage to a variety of things,
and most importantly, reduce or eliminate the chances of someone dying
from a wildfire. A fire detection system should be efficient and accurate
with respect to extinguishing wildfires in their earlier stages to prevent
the spread of them along with their consequences. Computer Vision is
potentially a more reliable, fast, and widespread method we need. The
current research in the field of preliminary fire detection has several prob-
lems related to unrepresentative data being used to train models and their
existing varied amounts of label imbalance in the classes (commonly fire
and non-fire) of their dataset. We propose a more representative and
evenly distributed data through better settings, lighting, atmospheres,
etc., and class distribution in the entire dataset. After thoroughly exam-
ining the results of this research, it can be inferred that they supported
the dataset’s strengths by being a viable resource when tested in the real
world on unfamiliar data. This is evident since as the model trains on
the dataset, it is able to generalize on it, hence confirming this is a viable
Machine Learning setting that has practical impact. Since severe droughts are occurring more frequently and lengthening the dry season in the Amazon
Rainforest, it is important to detect wildfires promptly and forecast possible spread for effective sup-
pression response. Though computer vision researchers have applied algorithms to automatically detect
wildfires, current models are computationally expensive and not versatile enough for the low technol-
ogy conditions of South American wildfire hotspots. This comprehensive deep learning study first
trains a Fully Convolutional Neural Network with skip connections on multispectral Landsat 8 images
of Ecuador and the Galapagos. The model uses Green and Short-wave Infrared (SWIR) bands as in-
puts to predict each image’s corresponding pixel-level binary fire mask. This model achieves a 0.962
validation F2 score and a 0.932 F2 score on test data from Guyana and Suriname. Afterward, image
segmentation is conducted on the Cirrus band using K-Means Clustering to simplify continuous pixel
values into three discrete classes representing differing degrees of cirrus cloud contamination. Two ad-
ditional Convolutional Neural Networks are trained to classify the presence of a wildfire using these
segmented cirrus images. The ”experimental” model trained on the segmented inputs and SWIR data
achieves a binary accuracy that is 2.306% higher than that of the ”benchmark model” that is trained
only on SWIR data. The difference in performance has a p-value of 0.00968. This proof of concept
reveals that feature simplification can improve the performance of wildfire detection models. Overall,
the software built in this study is useful for early and accurate detection of wildfires in South America.
The size and frequency of wildland fires in the western United States have dramatically
increased in recent years. On high-fire-risk days, a small fire ignition can rapidly grow and become out of
control. Early detection of fire ignitions from initial smoke can assist the response to such fires before they
become difficult to manage. Past deep learning approaches for wildfire smoke detection have suffered
from small or unreliable datasets that make it difficult to extrapolate performance to real-world scenarios.
In this work, we present the Fire Ignition Library (FIgLib), a publicly available dataset of nearly 25,000
labeled wildfire smoke images as seen from fixed-view cameras deployed in Southern California. We
also introduce SmokeyNet, a novel deep learning architecture using spatiotemporal information from
camera imagery for real-time wildfire smoke detection. When trained on the FIgLib dataset, SmokeyNet
outperforms comparable baselines and rivals human performance. We hope that the availability of the
FIgLib dataset and the SmokeyNet architecture will inspire further research into deep learning methods
for wildfire smoke detection, leading to automated notification systems that reduce the time to wildfire
response.
"""

Cosine Similarity

In [ ]:
def get_similarity(summary):

  count_vect = CountVectorizer()

  corpus = [summary, human_summary]

  X_train_counts = count_vect.fit_transform(corpus)

  pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['summary','human_summary'])

  vectorizer = TfidfVectorizer()

  trsfm=vectorizer.fit_transform(corpus)
  pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['summary','human_summary'])

  sim = cosine_similarity(trsfm[0:1], trsfm)

  return sim

In [ ]:
print("Cosine similarity of summary generated from heapq:",get_similarity(summary_lib))


Cosine similarity of summary generated from heapq: [[1.         0.79421573]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print("Cosine similarity of summary generated from own implementation:",get_similarity(summary))

Cosine similarity of summary generated from own implementation: [[1.         0.77623598]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Rouge 

In [ ]:
def get_rouge(summary):
  rouge = Rouge()
  return rouge.get_scores(summary, human_summary)

In [ ]:
# Rouge scores for summary implemented through spacy and heapq
get_rouge(summary_lib)

[{'rouge-1': {'f': 0.1851152222224703,
   'p': 0.10898576512455516,
   'r': 0.6140350877192983},
  'rouge-2': {'f': 0.06593781828523634,
   'p': 0.037179734155268734,
   'r': 0.2911010558069382},
  'rouge-l': {'f': 0.1775594987619489,
   'p': 0.1045373665480427,
   'r': 0.5889724310776943}}]

In [ ]:
# Rouge scores for summary of own implementation
get_rouge(summary)

[{'rouge-1': {'f': 0.2867579862354163,
   'p': 0.22557471264367815,
   'r': 0.39348370927318294},
  'rouge-2': {'f': 0.07632093495159836,
   'p': 0.056480811006517015,
   'r': 0.11764705882352941},
  'rouge-l': {'f': 0.24474885381532502,
   'p': 0.1925287356321839,
   'r': 0.3358395989974937}}]